In [2]:
import pandas as pd
import numpy as np

import gspread

import panel as pn
pn.extension()
import holoviews as hv
hv.extension('bokeh')
import hvplot.pandas
from datetime import datetime 
import bokeh

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [3]:
gc = gspread.service_account(filename="service_account.json")
sh = gc.open("statement_")

In [4]:
ws = sh.worksheet('Sheet1')
df = pd.DataFrame(ws.get_all_records())
df.head()

,Sl. No.,Transaction Date,Value Date,Description,Chq / Ref No.,Amount,Dr / Cr,Balance
0,1,2023-06-04,04-06-2023,Recd:IMPS/315518174849/AREIN GUPT/KKBK/X5042/self,IMPS-315518225217,6000.00,CR,6172.49
1,2,2023-06-04,04-06-2023,UPI/Apple Services/315550956885/MandateRequest,UPI-315581476813,271.00,DR,5901.49
2,3,2023-06-04,04-06-2023,UPI/Zomato Ltd/352148580079/OidZTDUPIC23E8D,UPI-315587182931,290.90,DR,5610.59
3,4,2023-06-05,05-06-2023,UPI/Zomato Ltd/352202886222/OidZTDUPIC231E5,UPI-315693606281,168.46,DR,5442.13
4,5,2023-06-08,08-06-2023,UPI/Zomato Ltd/315936272873/OidZTDUPIC23AD0,UPI-315989796261,321.70,DR,5120.43


In [5]:
df = df[['Transaction Date', 'Description', 'Amount']]
df['Description'] = df['Description'].map(str.lower)

df = df.rename(columns={'Transaction Date': 'Date'})

df['Category'] = 'unassigned'                       

df.head()

,Date,Description,Amount,Category
0,2023-06-04,recd:imps/315518174849/arein gupt/kkbk/x5042/self,6000.00,unassigned
1,2023-06-04,upi/apple services/315550956885/mandaterequest,271.00,unassigned
2,2023-06-04,upi/zomato ltd/352148580079/oidztdupic23e8d,290.90,unassigned
3,2023-06-05,upi/zomato ltd/352202886222/oidztdupic231e5,168.46,unassigned
4,2023-06-08,upi/zomato ltd/315936272873/oidztdupic23ad0,321.70,unassigned


In [6]:
#Food

df['Category'] = np.where(df['Description'].str.contains(
    'zomato|swiggy|dominos|pizza'), 
    'Food', df['Category'] )

#Entertainment

df['Category'] = np.where(df['Description'].str.contains(
    'bigtree'), 
    'Entertainment', df['Category'] )

#Personal

df['Category'] = np.where(df['Description'].str.contains(
    'vansh|rachit|priyanshu'), 
    'Personal', df['Category'] )

#Online Shopping

df['Category'] = np.where(df['Description'].str.contains(
    'flipkart|amazon|apple'), 
    'Online Shopping', df['Category'] )

#Deposits

df['Category'] = np.where(df['Description'].str.contains(
    'recd:imps'), 
    'Deposit', df['Category'] )

#Services and Misc

df['Category'] = np.where(df['Description'].str.contains(
    'mo|ajit|manoj|int.pd'), 
    'Services_Misc', df['Category'] ) 


df['Date'] = pd.to_datetime(df['Date'])

# Extract the month and year information
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
    
pd.options.display.max_rows = 999
df.head(200)


,Date,Description,Amount,Category,Month,Year
0,2023-06-04,recd:imps/315518174849/arein gupt/kkbk/x5042/self,6000.00,Deposit,6,2023
1,2023-06-04,upi/apple services/315550956885/mandaterequest,271.00,Online Shopping,6,2023
2,2023-06-04,upi/zomato ltd/352148580079/oidztdupic23e8d,290.90,Food,6,2023
3,2023-06-05,upi/zomato ltd/352202886222/oidztdupic231e5,168.46,Food,6,2023
4,2023-06-08,upi/zomato ltd/315936272873/oidztdupic23ad0,321.70,Food,6,2023
5,2023-06-10,upi/pizza hut/316172030750/pay via razorpa,562.01,Food,6,2023
6,2023-06-11,upi/zomato/316290409634/pay via razorpa,252.86,Food,6,2023
7,2023-06-12,upi/zomato online o/316373132391/pay via razorpa,171.61,Food,6,2023
8,2023-06-24,upi/bigtree enterta/317523261369/bms,1080.44,Entertainment,6,2023
9,2023-06-24,upi/vansh gulati/317551742760/upi,1040.69,Personal,6,2023


In [7]:
# to check unassigned

unassigned = df.loc[df['Category'] == 'unassigned']
unassigned

,Date,Description,Amount,Category,Month,Year


In [8]:
# Get the latest month and year
latest_month = df['Month'].max()
latest_year = df['Year'].max()

#filter
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

In [9]:
last_month_expenses = last_month_expenses.groupby('Category')['Amount'].sum().reset_index()

last_month_expenses['Amount']=last_month_expenses['Amount'].astype('str')
last_month_expenses['Amount']=last_month_expenses['Amount'].str.replace('-','')
last_month_expenses['Amount']=last_month_expenses['Amount'].astype('float')       

last_month_expenses = last_month_expenses[last_month_expenses["Category"].str.contains("Excluded|unassigned") == False]    
last_month_expenses = last_month_expenses.sort_values(by='Amount', ascending=False)    #sort 
last_month_expenses['Amount'] = last_month_expenses['Amount'].round().astype(int)      #round 

last_month_expenses


,Category,Amount
3,Personal,2854
1,Food,2833
2,Online Shopping,1897
0,Entertainment,1621
4,Services_Misc,1080


In [10]:
last_month_expenses_tot = last_month_expenses['Amount'].sum()
last_month_expenses_tot


10285

In [11]:
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name="Income", value="0")
recurring_expenses_widget = pn.widgets.TextInput(name="Recurring Expenses", value="0")
monthly_expenses_widget = pn.widgets.TextInput(name="Non-Recurring Expenses", value=str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput(name="Last Month's Savings", value="0")

income_widget.param.watch(calculate_difference, "value")
recurring_expenses_widget.param.watch(calculate_difference, "value")
monthly_expenses_widget.param.watch(calculate_difference, "value")

pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()

Launching server at http://localhost:57364


In [12]:
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x='Category', 
    y='Amount', 
    height=350, 
    width=850, 
    title="Last Month Expenses",
    ylim=(0, 4000))

last_month_expenses_chart

:Bars   [Category]   (Amount)

In [13]:
df['Date'] = pd.to_datetime(df['Date'])  
# extract the month and year from the 'Date' column and create a new column 'Month-Year'          
df['Month-Year'] = df['Date'].dt.to_period('M')    
monthly_expenses_trend_by_cat = df.groupby(['Month-Year', 'Category'])['Amount'].sum().reset_index()

monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('str')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].str.replace('-','')
monthly_expenses_trend_by_cat['Amount']=monthly_expenses_trend_by_cat['Amount'].astype('float')
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat["Category"].str.contains("Excluded") == False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='Amount', ascending=False)
monthly_expenses_trend_by_cat['Amount'] = monthly_expenses_trend_by_cat['Amount'].round().astype(int)
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str)
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.rename(columns={'Amount': 'Amount '})

monthly_expenses_trend_by_cat

,Month-Year,Category,Amount
0,2023-06,Deposit,22000
7,2023-07,Personal,2912
12,2023-08,Personal,2854
10,2023-08,Food,2833
5,2023-07,Food,2693
2,2023-06,Food,2187
11,2023-08,Online Shopping,1897
4,2023-06,Personal,1640
9,2023-08,Entertainment,1621
1,2023-06,Entertainment,1080


In [14]:
select_category1 = pn.widgets.Select(name='Select Category', options=[
    'All',
    'Food',
    'Online Shopping',
    'Entertainment',
    'Services_Misc',
    'Personal',
    'Deposit'
])

select_category1

BokehModel(combine_events=True, render_bundle={'docs_json': {'fdf58477-13ed-485e-99fa-7d7883a80e0f': {'version…

In [15]:

def plot_expenses(category):
    if category == 'All':
        plot_df = monthly_expenses_trend_by_cat.groupby('Month-Year').sum()
    else:
        plot_df = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat['Category'] == category].groupby('Month-Year').sum()
    plot = plot_df.hvplot.bar(x='Month-Year', y='Amount ')
    return plot


@pn.depends(select_category1.param.value)
def update_plot(category):
    plot = plot_expenses(category)
    return plot

# layout
monthly_expenses_trend_by_cat_chart = pn.Row(select_category1, update_plot)
monthly_expenses_trend_by_cat_chart[1].width = 600

monthly_expenses_trend_by_cat_chart

BokehModel(combine_events=True, render_bundle={'docs_json': {'ba0e6ebe-43e1-47ef-8d57-d587ac83156b': {'version…

In [16]:
df = df[['Date', 'Category', 'Description', 'Amount']]
df['Amount']=df['Amount'].astype('str')
df['Amount']=df['Amount'].str.replace('-','')
df['Amount']=df['Amount'].astype('float')        

df = df[df["Category"].str.contains("Excluded") == False]    
df['Amount'] = df['Amount'].round().astype(int)      
df

,Date,Category,Description,Amount
0,2023-06-04,Deposit,recd:imps/315518174849/arein gupt/kkbk/x5042/self,6000
1,2023-06-04,Online Shopping,upi/apple services/315550956885/mandaterequest,271
2,2023-06-04,Food,upi/zomato ltd/352148580079/oidztdupic23e8d,291
3,2023-06-05,Food,upi/zomato ltd/352202886222/oidztdupic231e5,168
4,2023-06-08,Food,upi/zomato ltd/315936272873/oidztdupic23ad0,322
5,2023-06-10,Food,upi/pizza hut/316172030750/pay via razorpa,562
6,2023-06-11,Food,upi/zomato/316290409634/pay via razorpa,253
7,2023-06-12,Food,upi/zomato online o/316373132391/pay via razorpa,172
8,2023-06-24,Entertainment,upi/bigtree enterta/317523261369/bms,1080
9,2023-06-24,Personal,upi/vansh gulati/317551742760/upi,1041


In [22]:

def filter_df(category):
    if category == 'All':
        return df
    return df[df['Category'] == category]


summary_table = pn.widgets.DataFrame(filter_df('All'), height = 300,width=400)


def update_summary_table(event):
    summary_table.value = filter_df(event.new)


select_category1.param.watch(update_summary_table, 'value')

summary_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'0c8fdfab-59a8-4611-a621-5a451ff23d1a': {'version…

In [24]:
template = pn.template.FastListTemplate(
    title="Personal Finances Summary",
    sidebar=[
        pn.pane.Markdown("## *Do not save what is left after spending, but spend what is left after saving*"),
        pn.pane.JPG('money_manage.jpg', sizing_mode='scale_both'),
        pn.pane.Markdown(""),
        pn.pane.Markdown(""),
        select_category1
    ],
    main=[
        pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget, width=950),
        pn.Row(last_month_expenses_chart, height=240),
        pn.GridBox(
            monthly_expenses_trend_by_cat_chart[1],
            summary_table,
            ncols=2,
            width=500,  
            align='start',
            sizing_mode='stretch_width'
        )
    ]
)

template.show()

Launching server at http://localhost:57800
